In [18]:
# Function to initialize the simplex
def initialize_simplex(x0, bounds):
    n = len(x0)
    simplex = np.zeros((n + 1, n))
    simplex[0] = x0

    for i in range(n):
        point = np.array(x0)
        point[i] = bounds[i][0]
        simplex[i + 1] = point

        point = np.array(x0)
        point[i] = bounds[i][1]
        simplex[i + 1] = point

    return simplex, bounds

In [19]:
# Function to sort simplex, calculate centroid, and perform reflection
def sort_and_reflect(simplex, fs, bounds):
    alpha=1.3487640718112321
    # Sort simplex according to function values
    idx = np.argsort(fs)
    simplex = simplex[idx]
    fs = fs[idx]

    # Calculate the centroid of the simplex
    xbar = np.mean(simplex[:-1], axis=0)

    # Reflection
    xr = xbar + alpha * (xbar - simplex[-1])
    xr = np.clip(xr, bounds[:, 0], bounds[:, 1])

    return simplex, fs, xbar, xr, bounds


In [38]:
# Function to handle reflection and expansion
def expansion_or_contraction(simplex, fs, xr, fxr, xbar, bounds):
    gamma=2.139744408219208
    rho=0.3740185211921033
    if fs[0] <= fxr < fs[-2]:
        # Successful reflection
        simplex[-1] = xr
        fs[-1] = fxr
        x = xr
        expansion = 1
        return simplex, fs, xr, fxr, x, expansion, bounds
    elif fxr < fs[0]:
        # Expansion
        xe = xbar + gamma * (xr - xbar)
        xe = np.clip(xe, bounds[:, 0], bounds[:, 1])
        expansion = 2
        x = xe
    else:
        # Contraction
        xc = xbar + rho * (simplex[-1] - xbar)
        xc = np.clip(xc, bounds[:, 0], bounds[:, 1])
        expansion = 3
        x = xc
    return simplex, fs, xr, fxr, x, expansion, bounds


In [21]:
def update_simplex_after_expansion_or_contraction(simplex, fs, xr, fxr, x, fxx, expansion, bounds):
    shrink = 0
    sigma=0.5501390442296267
    if expansion == 2:
        if fxx < fxr:
            simplex[-1] = x
            fs[-1] = fxx
        else:
            simplex[-1] = xr
            fs[-1] = fxr
        return simplex, fs, bounds, shrink
    elif expansion == 3:
        if fxx < fs[-1]:
            simplex[-1] = x
            fs[-1] = fxx
            return simplex, fs, bounds, shrink
        else:
            # Shrink
            simplex[1:] = simplex[0] + sigma * (simplex[1:] - simplex[0])
            shrink = 1
            return simplex, bounds, shrink

# alpha=1.3487640718112321, gamma=2.139744408219208, rho=0.3740185211921033, sigma=0.5501390442296267, tol=1e-4)

In [56]:
import numpy as np

# Define the objective function (you can replace this with any function you want)
def objective_function(x):
    return (x[0] - 2) ** 2 + (x[1] - 3) ** 2

# Define the initial guess and bounds
x0 = np.array([0.0, 0.0])
bounds = np.array([[-5.0, 5.0], [-5.0, 5.0]])

# Set algorithm parameters
maxiter = 50  # Run one iteration
# alpha = 1.0
# gamma = 2.0
# rho = 0.5
# sigma = 0.5
# tol = 1e-4

# Initialize the simplex
simplex, bounds = initialize_simplex(x0, bounds)

# Evaluate the objective function at each vertex of the simplex
fs = np.zeros(len(simplex))
for i, point in enumerate(simplex):
    fs[i] = objective_function(point)

# Run one iteration of the algorithm
for i in range(maxiter):
    # Sort simplex, calculate centroid, and perform reflection
    simplex, fs, xbar, xr, bounds = sort_and_reflect(simplex, fs, bounds)

    # Evaluate the objective function at the reflected point
    fxr = objective_function(xr)

    # Handle reflection and expansion
    simplex, fs, xr, fxr, x, expansion, bounds = expansion_or_contraction(simplex, fs, xr, fxr, xbar, bounds)
    
    fxx = objective_function(x)
    
    if expansion == 1:
        continue

    # Update the simplex after expansion or contraction
    simplex, fs, bounds, shrink = update_simplex_after_expansion_or_contraction(simplex, fs, xr, fxr, x, fxx, expansion, bounds)
    
    if shrink == 0:
        continue
        
    # Evaluate the objective function at the shrunk simplex
    for i, point in enumerate(simplex):
        fs[i] = objective_function(point)

    print(f'Iteration {i + 1}:')
    print(f'Simplex vertices: {simplex}')
    print(f'Function values: {fs}')
    print(f'Best function value: {fs[0]}\n')
    
print(f'Best point: {simplex[0]}')
print(f'Best function value: {fs[0]}\n')


Best point: [2.00000392 3.0000025 ]
Best function value: 2.1603313767433688e-11



In [60]:
print(x0)                  #start point
objective_function(x0)     #error of starting point

[0. 0.]


13.0

In [61]:
print(simplex[0])                     #optimized point
objective_function(simplex[0])        #error of optimized point

[2.00000392 3.0000025 ]


2.1603313767433688e-11